In [1]:
# Dependencies
# ----------------------------------
import pandas as pd

# Imports the method used for connecting to DBs
from sqlalchemy import create_engine

# Imports the methods needed to abstract classes into tables
from sqlalchemy.ext.declarative import declarative_base

# Allow us to declare column types
from sqlalchemy import Column, Integer, String, Float 

from sqlalchemy import create_engine, Column, Integer, String, ForeignKey
Base = declarative_base()

In [2]:
oscars_df = pd.read_csv('output/final_oscars.csv')
imdb_df = pd.read_csv('output/cleaned_imdb.csv')

In [3]:
# Create the Classes
class Directors(Base):
    __tablename__ = 'directors'
    director_id = Column(String, primary_key=True)
    director_name = Column(String(255))
    
class Movies(Base):
    __tablename__ = 'movies'
    movie_id = Column(String, primary_key=True)
    movie_title = Column(String(255))
    year = Column(Integer)
    director_id = Column(String(255), ForeignKey('directors.director_id'))

class Oscar_categories(Base):
    __tablename__ = 'oscar_categories'
    oscar_id = Column(String, primary_key=True)
    award_category = Column(String(255))
    
class Movie_awards(Base):
    __tablename__ = 'movie_awards'
    movie_id = Column(String, ForeignKey('movies.movie_id'), primary_key=True)
    oscar_id = Column(String(255), ForeignKey('oscar_categories.oscar_id'), primary_key=True)

class Directing_awards(Base):
    __tablename__ = 'directing_awards'
    director_id = Column(String, ForeignKey('directors.director_id'), primary_key=True)
    oscar_id = Column(String(255), ForeignKey('oscar_categories.oscar_id'))

In [4]:
# These tables only exists in python and not in the actual database
Base.metadata.tables

immutabledict({'directors': Table('directors', MetaData(bind=None), Column('director_id', String(), table=<directors>, primary_key=True, nullable=False), Column('director_name', String(length=255), table=<directors>), schema=None), 'movies': Table('movies', MetaData(bind=None), Column('movie_id', String(), table=<movies>, primary_key=True, nullable=False), Column('movie_title', String(length=255), table=<movies>), Column('year', Integer(), table=<movies>), Column('director_id', String(length=255), ForeignKey('directors.director_id'), table=<movies>), schema=None), 'oscar_categories': Table('oscar_categories', MetaData(bind=None), Column('oscar_id', String(), table=<oscar_categories>, primary_key=True, nullable=False), Column('award_category', String(length=255), table=<oscar_categories>), schema=None), 'movie_awards': Table('movie_awards', MetaData(bind=None), Column('movie_id', String(), ForeignKey('movies.movie_id'), table=<movie_awards>, primary_key=True, nullable=False), Column('os

In [5]:
database = "movies_db"
username = 'enter username'
password = 'enter password'
host = 'localhost'
port = 5432

In [6]:
# Create our database engine
engine = create_engine(f"postgresql://{username}:{password}@{host}:{port}/{database}") 

In [7]:
# This is where we create our tables in the database
Base.metadata.create_all(engine)

In [8]:
# The ORM’s “handle” to the database is the Session.
from sqlalchemy.orm import Session
session = Session(engine)

In [9]:
# Read csv for directors
directors_df = pd.read_csv('output/directors.csv',  index_col=0)
directors_df.head()

,director,director_id
0,Alexander Black,0
1,Charles Tait,1
2,Urban Gad,2
3,Charles L. Gaskill,3
4,Francesco Bertolini,4


In [10]:
# Note that adding to the session does not update the table. It queues up those queries.
# Adding data to directors table from directors csv file
for index, name in directors_df.iterrows():
    session.add(Directors(director_id=name['director_id'], director_name=name['director']))

In [11]:
# session.new

In [12]:
session.commit()

In [13]:
imdb_df = pd.read_csv('output/cleaned_imdb.csv',  index_col=0)
imdb_df.head()

,imdb_title_id,title,year,genre,country,director,writer,production_company,director_id
0,tt0000009,Miss Jerry,1894,Romance,USA,Alexander Black,Alexander Black,Alexander Black Photoplays,0
1,tt0000574,The Story of the Kelly Gang,1906,"Biography, Crime, Drama",Australia,Charles Tait,Charles Tait,J. and N. Tait,1
2,tt0001892,Den sorte drøm,1911,Drama,"Germany, Denmark",Urban Gad,"Urban Gad, Gebhard Schätzler-Perasini",Fotorama,2
3,tt0002101,Cleopatra,1912,"Drama, History",USA,Charles L. Gaskill,Victorien Sardou,Helen Gardner Picture Players,3
4,tt0002130,L'Inferno,1911,"Adventure, Drama, Fantasy",Italy,Francesco Bertolini,Dante Alighieri,Milano Film,4


In [14]:
# Adding data to movies table from imdb csv file
for index, name in imdb_df.iterrows():
    session.add(Movies(movie_id=name['imdb_title_id'], movie_title=name['title'], year=name['year'], director_id=name['director_id']))

In [15]:
# session.new

In [16]:
session.commit()

In [17]:
# Read csv for oscars categories
oscars_categories = pd.read_csv('output/oscars_categories.csv',  index_col=0)
oscars_categories.head()

,categories,id
0,DIRECTING,0
1,CINEMATOGRAPHY (Black-and-White),1
2,CINEMATOGRAPHY (Color),2
3,BEST MOTION PICTURE,3
4,BEST PICTURE,4


In [18]:
# Adding data to oscars categories table from csv file
count = 0
for index, name in oscars_categories.iterrows():
    session.add(Oscar_categories(oscar_id=count, award_category=name['categories']))
    count +=1

In [19]:
session.commit()

In [20]:
# Read csv for movie awards
movie_awards = pd.read_csv('output/movie_awards.csv',  index_col=0)
movie_awards.head()

,title_id,oscar_id
0,tt0093351,1
1,tt1781058,2
2,tt0435669,1
3,tt0026402,3
4,tt0116477,3


In [21]:
# Adding data to movie_awards relationship table from csv file
for index, name in movie_awards.iterrows():
    session.add(Movie_awards(movie_id=name['title_id'], oscar_id=name['oscar_id']))

In [22]:
session.commit()

In [23]:
# Read csv for directing awards
directing_awards = pd.read_csv('output/directing_awards.csv',  index_col=0)
directing_awards.head(5)

,oscar_id,director_id
0,0,13608
1,0,9795
2,0,8333
3,0,12401
4,0,10621


In [24]:
# Adding data to directing_awards relationship table from csv file
for index, name in directing_awards.iterrows():
    session.add(Directing_awards(director_id=name['director_id'], oscar_id=name['oscar_id']))

In [ ]:
# session.rollback()

In [25]:
import numpy
from psycopg2.extensions import register_adapter, AsIs
def addapt_numpy_float64(numpy_float64):
    return AsIs(numpy_float64)
def addapt_numpy_int64(numpy_int64):
    return AsIs(numpy_int64)
register_adapter(numpy.float64, addapt_numpy_float64)
register_adapter(numpy.int64, addapt_numpy_int64)

In [26]:
session.commit()

In [27]:
session.close()